In [1]:
import collections
import requests
import json
import re
import os
import sys

with open('../../data/speeches_20.jsonl', 'r', encoding = 'utf8') as fp:
    data = list(fp)
speeches = []
for line in data:
    speeches.append(json.loads(line))

In [2]:
# globale items
regex = "{(.*?)}" # hier die genaue Definition der Kommentare vom Parser <<< >>>
kommentare = []

In [8]:
#def groupSpeechesByParty(speeches):
#    groupedSpeeches = {}
#    for speech in speeches:
#        tpo = speech['party']
#        if tpo in groupedSpeeches:
#            groupedSpeeches[tpo].append(speech)
#        else:
#            groupedSpeeches[tpo] = [speech]
#    return groupedSpeeches

In [9]:
def clean_partyNames(speeches):
    for speech in speeches:
        if 'BÜNDNIS\xa090/DIE GRÜNEN' in speech['party']:
            speech.update({'party': 'BÜNDNIS 90/DIE GRÜNEN'})
#print(speech['party'])

In [10]:
groupbyparty = groupSpeechesByParty(speeches)
groupbyparty.keys()
print(len(speeches)) # Anzahl aller Reden
for ele in groupbyparty:
    print(len(groupbyparty[ele]))
# Überprüfung mit bspw.: len(groupbyparty['SPD'])

2777
698
607
354
395
353
241
26
103


In [11]:
for speech in speeches:
    if 'BÜNDNIS\xa090/DIE GRÜNEN' in speech['party']:
        speech.update({'party': 'DIE GRÜNEN'})
groupbyparty.keys()

dict_keys(['SPD', 'CDU/CSU', 'DIE GRÜNEN', 'FDP', 'AfD', 'DIE LINKE', 'Fraktionslos', 'BÜNDNIS 90/DIE GRÜNEN'])

In [18]:
# Funktion: speeches_comments
def extract_comments(speeches):
    comments = []
    modified_speeches = []
    for speech in speeches:
        text = speech['text']
        matches = re.findall(regex,text)
        speech['comments'] = matches
        modified_speeches.append(speech)
    return modified_speeches
# Funktionsaufruf:
speeches_comments = extract_comments(speeches)
# Funktionsausgabe: print(speeches_comments[0]['comments'])
#                   print(speeches_comments[0]['party'])

def filter_with_complex_non_words(search_terms,non_words,speeches):
    filtered_speeches = []
    for speech in speeches:
        for comment in speech['comments']:
            if all([term in comment for term in search_terms]) and not any([term in comment for term in non_words]):
                filtered_speeches.append(speech)
                break
    return filtered_speeches
# Funktionsaufruf:  specific_and_non_search = filter_with_complex_non_words(['Beifall','CDU'],['Zuruf'],speeches_comments)
# Funktionsausgabe: print(specific_and_non_search[0])                               enthält AND enthält nicht
#                   print(len(specific_and_non_search))

In [35]:
specific_and_non_search = filter_with_complex_non_words(['DIE LINKE'],[],speeches_comments)
print(f'Alle Kommentare an denen Die Linke beiteiligt war:            {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['SPD'],[],speeches_comments)
print(f'Alle Kommentare an denen SPD beiteiligt war:                  {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['BÜNDNIS 90/DIE GRÜNEN'],[],speeches_comments)
print(f'Alle Kommentare an denen BÜNDNIS 90/DIE GRÜNE beiteiligt war: {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['CDU/CSU'],[],speeches_comments)
print(f'Alle Kommentare an denen CDU/CSU beiteiligt war:              {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['FDP'],[],speeches_comments)
print(f'Alle Kommentare an denen FDP beiteiligt war:                  {len(specific_and_non_search)}')
specific_and_non_search = filter_with_complex_non_words(['AfD'],[],speeches_comments)
print(f'Alle Kommentare an denen AfD beiteiligt war:                  {len(specific_and_non_search)}')

Alle Kommentare an denen Die Linke beiteiligt war:            482
Alle Kommentare an denen SPD beiteiligt war:                  2326
Alle Kommentare an denen BÜNDNIS 90/DIE GRÜNE beiteiligt war: 1635
Alle Kommentare an denen CDU/CSU beiteiligt war:              1718
Alle Kommentare an denen FDP beiteiligt war:                  2215
Alle Kommentare an denen AfD beiteiligt war:                  1320
